# PCA Strategy: Flatteners and Steepeners

Given the PCA Var-Cov decomposition, we are going to conduct a strategy that is neutral to changes in the level of the curve den delta neutral. 

Our PCA decompisition looks like the following:

$$  F_t = \Delta y_{t} E$$
$$ \Delta y_{t} = F_t E^T$$ 

Where:

- $\Delta y_t$ is $(n, m)$ matrix with $n =$ number of periods in-sample and $m$ maturities in our yield curve (30).
- $F_t$ is the  $(n, f)$ matrix  of series of PCA scores, with $f$ number of factors to project (3 in our baseline case, ~97\% variance explained)
- $E$ $(f, m)$ matrix of eigenvectors of the decomposition of $\Sigma$, i.e. the loading of each PCA factor of each maturity.   
